In [1]:
from urllib.request import urlopen
import os
from bs4 import BeautifulSoup as BeautifulSoup
import pandas as pd
import nltk
from pandas.core.frame import DataFrame
import sys
import numpy as np

In [2]:
# Step 1: Define funtions to download filings
def get_list(ticker, file_type, todate):

    base_url_part1 = "http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK="
    base_url_part2 = "&type=&dateb=&owner=&start="
    base_url_part3 = "&count=100&output=xml"
    href = []
    
    for page_number in range(0,2000,100):
    
        base_url = base_url_part1 + ticker + base_url_part2 + str(page_number) + base_url_part3
        
        sec_page = urlopen(base_url)
        sec_soup = BeautifulSoup(sec_page)
        
        filings = sec_soup.findAll('filing')
        
        for filing in filings:
            report_year = int(filing.datefiled.get_text()[0:4])
            if (filing.type.get_text() == file_type) & (report_year > todate):
                print (filing.filinghref.get_text())
                href.append(filing.filinghref.get_text())
    
    return href

pre_url=[]
def download_report(url_list,dir_path, typefile):
    global pre_url
    target_base_url = 'http://www.sec.gov'
    
    # type = 'EX-101.INS'
    target_file_type = typefile
    
    for report_url in url_list:
        report_page = urlopen(report_url)
        report_soup = BeautifulSoup(report_page)
        
        xbrl_file = report_soup.findAll('tr')
        #print (xbrl_file)
        for item in xbrl_file:
            try:
                if item.findAll('td')[3].get_text() == target_file_type:
                    if not os.path.exists(dir_path):
                        os.makedirs(dir_path)
                    
                    end_path=item.findAll('td')[2].find('a')['href']
                    end_path=end_path.split('/')
                    #print (end_path[1])
                    if end_path[1]=='ix?doc=':
                        del end_path[1]
                    end_path=','.join(end_path)
                    end_path=end_path.replace(',', '/')
                    #print (end_path)
                    target_url = target_base_url + end_path
                    pre_url.append(target_url)
                    #print ("Target URL found!")
                    #print ("Target URL is:", target_url)
                    
                    file_name = target_url.split('/')[-1]
                    print (file_name)
                   
                    xbrl_report = urlopen(target_url)
                    output = open(os.path.join(dir_path,file_name),'wb')
                    output.write(xbrl_report.read())
                    output.close()
                    
            except:
                pass
#     return pre_url

In [3]:
# Step 2: Define funtions to download filings
# Import tickers
# TickerFile = pd.read_csv("companylist.csv")
# Tickers = TickerFile['Symbol'].tolist()

# tickers = Tickers

# for ticker in tickers:

def getfiles(tickername, filename, year):   
    ticker=tickername
    url_list= get_list(ticker, filename,year-1)
    base_path = "C:/data/CS/project3"
    dir_path = base_path + "/"+ticker+"/"+filename
    download_report(url_list,dir_path, filename)
#     url_to_text=download_report(url_list,dir_path, filename)
#     print (url_to_text)
    
#     url=url_to_text[0]
#     html = urlopen(url).read()
#     soup = BeautifulSoup(html)

#     # kill all script and style elements
#     for script in soup(["script", "style"]):
#         script.extract()    # rip it out

#     # get text
#     text = soup.get_text()

#     # break into lines and remove leading and trailing space on each
#     lines = (line.strip() for line in text.splitlines())
#     # break multi-headlines into a line each
#     chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
#     # drop blank lines
#     text = '\n'.join(chunk for chunk in chunks if chunk)
#     print (text)
    

In [4]:
#getfiles ('ETSY','8-K', 2019)
getfiles('ETSY', '10-Q', 2019)

https://www.sec.gov/Archives/edgar/data/1370637/000137063719000146/0001370637-19-000146-index.htm
https://www.sec.gov/Archives/edgar/data/1370637/000137063719000090/0001370637-19-000090-index.htm
etsy6301910q.htm
etsy3311910q.htm


In [5]:
pre_url

['http://www.sec.gov/Archives/edgar/data/1370637/000137063719000146/etsy6301910q.htm',
 'http://www.sec.gov/Archives/edgar/data/1370637/000137063719000090/etsy3311910q.htm']

In [6]:
def sift_tables(html):
    from bs4 import BeautifulSoup
    import csv
    html = urlopen(html)
    soup = BeautifulSoup(html)
    table = soup.findAll("table")
    #print (table)
    #output_rows = []
    output_row = []
    for a in table:
        for b in a.findAll('tr'):
            #print (b)
            for c in b.findAll('td'):
                for d in c.findAll('span'):                   
                    output_row.append(d.get_text())
#     output_row = np.array(output_row)
#     output_row=output_row.map(lambda x: [word.lower() for word in x])
    output=[]
    for str in output_row:
        output.append(str.lower())
    return  output
    
# output_row
# for i, keyword in enumerate (output_row):
#     if 'Volatility' in keyword :
        
#         print (output_row[i+1])

In [7]:
def find_values(hotword, file_url):
    table_list=sift_tables(file_url)
    c={}
    c_dict=[]
    #print (table_list)
    for i, keyword in enumerate (table_list):
        if hotword in keyword :
            temp=table_list[i+1]
            temp=temp.replace(',','')
            try:
                temp=int(temp)
                
                #print(c)
                c_dict.append(temp)
            except:
                c[hotword]=temp
    if c_dict:
        c[hotword]=c_dict

    if c=={}:
        c[hotword]=None
        
   
    return c
            

In [8]:
find_values('shares', pre_url[0])

{'shares': [120, 159403, 247941, 104849, 253989, 120198526, 130807743, 153865]}

In [9]:
final_table={}
hotword_list=['volatility', 'notional', 'strike percentage', 'expire', 'expiry', 'shares', 'premium', 'conversion rate']

for word in hotword_list:
    final_table.update(find_values(word, pre_url[0]))
    # final_table.append(aa)
    # final_table
print (final_table)
df=pd.DataFrame(columns=['Parameters','Values'], index=None)
values=[]
for key in final_table:
    values.append(final_table[key])
df['Values']=values
df['Parameters']=hotword_list

df.to_csv('C:/data/CS/project3/par_val.csv', index=None)
df

{'volatility': '39.2% - 39.5%', 'notional': None, 'strike percentage': None, 'expire': None, 'expiry': None, 'shares': [120, 159403, 247941, 104849, 253989, 120198526, 130807743, 153865], 'premium': None, 'conversion rate': None}


,Parameters,Values
0,volatility,39.2% - 39.5%
1,notional,None
2,strike percentage,None
3,expire,None
4,expiry,None
5,shares,"[120, 159403, 247941, 104849, 253989, 12019852..."
6,premium,None
7,conversion rate,None
